###  **Module imports**

In [0]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import scipy as sp
import h5py as h5
from scipy import io
from scipy.misc import imsave
from scipy import interpolate
from scipy import ndimage
from scipy.io import savemat, loadmat
from os import listdir
import random
import os

import matplotlib.pyplot as plt
from skimage.measure import compare_ssim as ssim

import sys
sys.path.insert(0, '/media/flash/ExTra/ulf_focdef/tools/')

###  **Light field parameters**

In [0]:
#parameters
lfsize = [375, 540, 7, 7] #dimensions of Lytro light fields
patchsize = [210, 210] #spatial dimensions of training light fields
disp_mult = 10.0 #max disparity between adjacent veiws

In [0]:
def getLf(data,lfsize):
    B,c,H,W = data.shape
    v = lfsize[3]
    u = lfsize[2]
    lf = data.reshape(B,-1,3,W,H)
    lf = lf.reshape(B,v,u,3,W,H)
    lf = lf.transpose(0,5,4,1,2,3) # B, H, W, v, u, 3

    return lf

def lfsave(path,idx,key,img):
    path = path + str(idx).zfill(3)
    if not os.path.isdir(path):
        os.mkdir(path)

    path = path+'/'+key+'_'
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            imsave(path+str(i)+str(j)+'.png',img[i,j])

###  **Forward warping of light field**

In [0]:
#render light field from input image and ray depths

def depth_rendering(central, ray_depths, lfsize):
    with tf.variable_scope('depth_rendering') as scope:
        b_sz = tf.shape(central)[0]
        y_sz = tf.shape(central)[1]
        x_sz = tf.shape(central)[2]
        u_sz = lfsize[2]
        v_sz = lfsize[3]
        
        central = tf.expand_dims(tf.expand_dims(central, 3), 4)
                                                
        #create and reparameterize light field grid
        b_vals = tf.to_float(tf.range(b_sz))
        v_vals = tf.to_float(tf.range(v_sz)) - tf.to_float(v_sz/2)
        u_vals = tf.to_float(tf.range(u_sz)) - tf.to_float(u_sz/2)
        y_vals = tf.to_float(tf.range(y_sz))
        x_vals = tf.to_float(tf.range(x_sz))
    
        b, y, x, v, u = tf.meshgrid(b_vals, y_vals, x_vals, v_vals, u_vals, indexing='ij')
               
        #warp coordinates by ray depths
        y_t = y + v * ray_depths
        x_t = x + u * ray_depths
        
        v_r = tf.zeros_like(b)
        u_r = tf.zeros_like(b)
        
        #indices for linear interpolation
        b_1 = tf.to_int32(b)
        y_1 = tf.to_int32(tf.floor(y_t))
        y_2 = y_1 + 1
        x_1 = tf.to_int32(tf.floor(x_t))
        x_2 = x_1 + 1
        v_1 = tf.to_int32(v_r)
        u_1 = tf.to_int32(u_r)
        
        y_1 = tf.clip_by_value(y_1, 0, y_sz-1)
        y_2 = tf.clip_by_value(y_2, 0, y_sz-1)
        x_1 = tf.clip_by_value(x_1, 0, x_sz-1)
        x_2 = tf.clip_by_value(x_2, 0, x_sz-1)
        
        #assemble interpolation indices
        interp_pts_1 = tf.stack([b_1, y_1, x_1, v_1, u_1], -1)
        interp_pts_2 = tf.stack([b_1, y_2, x_1, v_1, u_1], -1)
        interp_pts_3 = tf.stack([b_1, y_1, x_2, v_1, u_1], -1)
        interp_pts_4 = tf.stack([b_1, y_2, x_2, v_1, u_1], -1)
        
        #gather light fields to be interpolated
        lf_1 = tf.gather_nd(central, interp_pts_1)
        lf_2 = tf.gather_nd(central, interp_pts_2)
        lf_3 = tf.gather_nd(central, interp_pts_3)
        lf_4 = tf.gather_nd(central, interp_pts_4)
        
        #calculate interpolation weights
        y_1_f = tf.to_float(y_1)
        x_1_f = tf.to_float(x_1)
        d_y_1 = 1.0 - (y_t - y_1_f)
        d_y_2 = 1.0 - d_y_1
        d_x_1 = 1.0 - (x_t - x_1_f)
        d_x_2 = 1.0 - d_x_1
        
        w1 = d_y_1 * d_x_1
        w2 = d_y_2 * d_x_1
        w3 = d_y_1 * d_x_2
        w4 = d_y_2 * d_x_2
        
        lf = tf.add_n([w1*lf_1, w2*lf_2, w3*lf_3, w4*lf_4])
                        
    return lf

###  **Functions for CNN layers**

In [0]:

def weight_variable(w_shape, name):
    return tf.get_variable(name, w_shape, initializer=tf.contrib.layers.xavier_initializer_conv2d())

def bias_variable(b_shape, init_bias=0.0):
    return tf.get_variable('bias', b_shape, initializer=tf.constant_initializer(init_bias))

#CNN layer no activation
def cnn_layer_no_act(input_tensor, w_shape, b_shape, layer_name, is_training, rate=1, padding_type='SAME'):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape, '_weights')
        h = tf.nn.atrous_conv2d(input_tensor, W, rate, padding=padding_type, name=layer_name + '_conv')
        h = h + bias_variable(b_shape)
        return h

#Batch normalization 
def bn(input_tensor, is_training, name='instance_normalization'):
    with tf.variable_scope(name):
        depth = input_tensor.get_shape()[-1]
        scale = tf.get_variable(
            'scale', [depth],
            initializer=tf.random_normal_initializer(1.0, 0.02, dtype=tf.float32))
        offset = tf.get_variable(
            'offset', [depth], initializer=tf.constant_initializer(0.0))
        mean, variance = tf.nn.moments(input_tensor, axes=[1, 2], keep_dims=True)
        epsilon = 1e-5
        inv = tf.rsqrt(variance + epsilon)
        normalized = (input_tensor - mean) * inv
        return scale * normalized + offset
    
#standard atrous layer
def cnn_layer(input_tensor, w_shape, b_shape, layer_name, is_training, rate=1, padding_type='SAME'):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape, '_weights')
        h = tf.nn.atrous_conv2d(input_tensor, W, rate, padding=padding_type, name=layer_name + '_conv')
        h = h + bias_variable(b_shape)
        h = tf.nn.elu(h)
        h = tf.contrib.layers.batch_norm(h, scale=True, updates_collections=None, 
                                             is_training=is_training, scope=layer_name + '_bn')
        #h = bn(h, is_training=is_training, name=layer_name + '_bn')
        return h
    
#layer with no normalization or activation
def cnn_layer_no_bn(input_tensor, w_shape, b_shape, layer_name, stride=1, padding_type='SAME'):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape, '_weights')
        h = tf.nn.conv2d(input_tensor, W, strides=[1, stride, stride, 1], padding=padding_type, name=layer_name + '_conv')
        h = h + bias_variable(b_shape)
        return h

#CNN layer with activation no batch norm
def cnn_layer_act_nobn(input_tensor, w_shape, b_shape, layer_name, is_training, rate=1, padding_type='SAME'):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape, '_weights')
        h = tf.nn.atrous_conv2d(input_tensor, W, rate, padding=padding_type, name=layer_name + '_conv')
        h = h + bias_variable(b_shape)
        h = tf.nn.elu(h)
        return h

#3D convolutional layer with activation and batch norm
def cnn_layer3D(input_tensor, w_shape, b_shape, layer_name, is_training, padding_type='SAME'):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape, '_weights')
        h = tf.nn.conv3d(input_tensor, W, strides=[1,1,1,1,1], padding=padding_type, name=layer_name + '_conv')
        h = h + bias_variable(b_shape)
        h = tf.nn.elu(h)
        h = tf.contrib.layers.batch_norm(h, scale=True, updates_collections=None, 
                                             is_training=is_training, scope=layer_name + '_bn')
        #h = bn(h, is_training=is_training, name=layer_name + '_bn')
        return h

#3D convolutional layer no activation no batch norm
def cnn_layer3D_no_bn(input_tensor, w_shape, b_shape, layer_name, padding_type='SAME'):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape, '_weights')
        h = tf.nn.conv3d(input_tensor, W, strides=[1,1,1,1,1], padding=padding_type, name=layer_name + '_conv')
        h = h + bias_variable(b_shape)
        return h

###  **Depth estimation network**

In [0]:
def depth_network(x, xc, lfsize, disp_mult, is_training, name):
    with tf.variable_scope(name):
        
        b_sz = tf.shape(x)[0]
        y_sz = tf.shape(x)[1]
        x_sz = tf.shape(x)[2]
        v_sz = lfsize[2]
        u_sz = lfsize[3]
        
        net_in = tf.concat([x,xc],axis=3)
        c1 = cnn_layer(net_in, [3, 3, 6, 16], [16], 'c1', is_training)
        c2 = cnn_layer(c1, [3, 3, 16, 64], [64], 'c2', is_training)
        c3 = cnn_layer(c2, [3, 3, 64, 128], [128], 'c3', is_training)
        c4 = cnn_layer(c3, [3, 3, 128, 128], [128], 'c4', is_training, rate=2)
        c5 = cnn_layer(c4, [3, 3, 128, 128], [128], 'c5', is_training, rate=2)
        c6 = cnn_layer(c5, [3, 3, 128, 128], [128], 'c6', is_training, rate=4)
        c7 = cnn_layer(c6, [3, 3, 128, 128], [128], 'c7', is_training, rate=8)
        c8 = cnn_layer(c7, [3, 3, 128, 64], [64], 'c8', is_training, rate=16)

        sc1 = cnn_layer(c3, [3, 3, 128, 128], [128], 'sc1', is_training)
        sc2 = cnn_layer(sc1, [3, 3, 128, 128], [128], 'sc2', is_training)
        sc3 = cnn_layer(sc2, [3, 3, 128, 64], [64], 'sc3', is_training)
        
        dsc1 = cnn_layer(c6, [3, 3, 128, 64], [64], 'dsc1', is_training)
        dsc2 = cnn_layer(dsc1, [3, 3, 64, 64], [64], 'dsc2', is_training)
        
        dsc3 = cnn_layer(c7, [3, 3, 128, 32], [32], 'dsc3', is_training)        
        dsc4 = cnn_layer(dsc3, [3, 3, 32, 32], [32], 'dsc4', is_training)        
        
        dsc5 = cnn_layer(c8, [3, 3, 64, 32], [32], 'dsc5', is_training)        
        dsc6 = cnn_layer(dsc5, [3, 3, 32, 32], [32], 'dsc6', is_training)        
        
        concat_feat = tf.concat([sc3,dsc2,dsc4,dsc6],axis=3)
        
        c13 = cnn_layer(concat_feat, [3, 3, 192, 128], [128], 'c13', is_training)
        c14 = cnn_layer(c13, [3, 3, 128, 128], [128], 'c14', is_training)
        c15 = cnn_layer(c14, [3, 3, 128, 49], [49], 'c15', is_training)
        c12 = cnn_layer(c15, [3, 3, 49, 49], [49], 'c12', is_training)
        c16 = cnn_layer(c12, [3, 3, 49, lfsize[2]*lfsize[3]], [lfsize[2]*lfsize[3]], 'c16', is_training)
        c17 = disp_mult*tf.tanh(cnn_layer_no_bn(c16, [3, 3, lfsize[2]*lfsize[3], lfsize[2]*lfsize[3]], 
                                                [lfsize[2]*lfsize[3]], 'c10'))
        
        return tf.reshape(c17, [b_sz, y_sz, x_sz, v_sz, u_sz], name='rayd')

###  **Light field refinement network**

In [0]:
def occlusions_network3D_2(d, xc, shear, lfsize, is_training, name):
    with tf.variable_scope(name):
        
        b_sz = tf.shape(d)[0]
        y_sz = tf.shape(d)[1]
        x_sz = tf.shape(d)[2]
        v_sz = lfsize[2]
        u_sz = lfsize[3]
        #depth
        d = tf.tile(tf.expand_dims(d,5), [1,1,1,1,1,3])
        d = tf.reshape(d, [b_sz, y_sz, x_sz, v_sz*u_sz, 3])
        #light field
        x = tf.reshape(shear, [b_sz, y_sz, x_sz, v_sz*u_sz, 3])
        #defocused image
        xc = tf.expand_dims(xc,3)
        #concatenated light field, depth and defocused image as input
        xdc = tf.concat([x, d, xc],axis=3)
        
        
        xdc = tf.transpose(xdc, [0, 4, 1, 2, 3]) # B, C, H, W, 2*v*u + 1
        #shear = tf.reshape(shear, [b_sz, y_sz, x_sz, v_sz*u_sz*3])
        #[filter_depth, filter_height, filter_width, in_channels, out_channels]
        c1 = cnn_layer3D(xdc, [3,3,3,99,98], [98], 'c1', is_training, padding_type='SAME')
        c2 = cnn_layer3D(c1, [3,3,3,98,98], [98], 'c2', is_training, padding_type='SAME')
        c3 = cnn_layer3D(c2, [3,3,3,98,49], [49], 'c3', is_training, padding_type='SAME')
        c6 = cnn_layer3D_no_bn(c3, [3, 3, 3, v_sz*u_sz, v_sz*u_sz], [v_sz*u_sz], 'c6', padding_type='SAME')
        # o - b,3,h,w,49
        c7 = tf.transpose(tf.reshape(c6, [b_sz, 3, y_sz, x_sz, v_sz, u_sz]), [0, 2, 3, 4, 5, 1])
        c8 = tf.sigmoid(c7 + shear)
        
        return tf.reshape(c8, [b_sz, y_sz, x_sz, v_sz, u_sz, 3])

###  **Pipeline**

In [0]:
#full forward model
def forward_model(x, xc, lfsize, disp_mult, is_training):
    with tf.variable_scope('forward_model', reuse=None) as scope:
        #predict ray depths from input image
        ray_depths = depth_network(x, xc, lfsize, disp_mult, is_training, 'ray_depths')

        #shear input image by predicted ray depths to render Lambertian light field
        lf_shear_r = depth_rendering(x[:, :, :, 0], ray_depths, lfsize)
        lf_shear_g = depth_rendering(x[:, :, :, 1], ray_depths, lfsize)
        lf_shear_b = depth_rendering(x[:, :, :, 2], ray_depths, lfsize)
        lf_shear = tf.stack([lf_shear_r, lf_shear_g, lf_shear_b], axis=5)

        #occlusion/non-Lambertian prediction network
        d = tf.stop_gradient(ray_depths)
        lfs = tf.stop_gradient(lf_shear)
        #y, yr = occlusions_network3D_1(d, xc, lfs, lfsize, is_training, 'occlusions')#occlusions
        y = occlusions_network3D_2(d, xc, lfs, lfsize, is_training, 'occ2')#occlusions
        return ray_depths, lf_shear, y, y

In [0]:
def normalize_lf(lf):
    return lf 

In [0]:
is_training = tf.placeholder(tf.bool, [],name='trainswitch')
x_batch = tf.placeholder(tf.float32,[None,patchsize[0],patchsize[1],3],name='cv')
xc_batch = tf.placeholder(tf.float32,[None,patchsize[0],patchsize[1],3],name='coded')
lf_batch = tf.placeholder(tf.float32,[None,patchsize[0],patchsize[1],lfsize[2],lfsize[3],3],name='lf')

batchsize = 1
code = np.ones([1,1,7,7,1])
code = code[np.newaxis,:,:,:,:,:]/49.0
code = np.tile(code,(1,patchsize[0],patchsize[1],1,1,3))

print(code.shape)


(1, 210, 210, 7, 7, 3)


###  **Forward model**

In [0]:
ray_depths, lf_shear, y, yr = forward_model(x_batch, xc_batch, lfsize, disp_mult, is_training)

#losses
with tf.name_scope('loss'):
    shear_loss = tf.reduce_mean(tf.abs(lf_shear-lf_batch))
    output_loss = tf.reduce_mean(tf.abs(y-lf_batch)) 
    train_loss = shear_loss + output_loss


###  **PSNR and SSIM evaluation**

In [0]:
def eval_ssim(refLf, genLf):
    ss = np.zeros([refLf.shape[0], refLf.shape[1], refLf.shape[2]])

    for i in range(refLf.shape[1]):
        for j in range(refLf.shape[2]):
            for k in range(refLf.shape[0]):
                refimg = (refLf[k,i,j,:,:,:]*255).astype(np.uint8)
                genimg = (genLf[k,i,j,:,:,:]*255).astype(np.uint8)
                #print refimg., genimg.astype(float64)
                #print type(refimg), type(genimg), refimg.shape, genimg.shape
                ss[k,i,j] = ssim(refimg, genimg, data_range=255.0, multichannel=True)
    ss = ss.sum(0)/float(refLf.shape[0])
    return ss

def psnr_eval(gt,rec):
    gt =  (gt*255).astype(np.uint8)
    rec = (rec*255).astype(np.uint8)
    # v,u,h,w,3
    psnr_val = -10*np.log10(np.mean(pow(gt-rec,2), axis=(2,3,4))) + 20*np.log10(255.0) # v,u
    psnr_val[3,3] = 0
    return psnr_val

def all_eval(Lf, lf_rec, lf_y):

    plf = psnr_eval(Lf,lf_rec)
    plfy = psnr_eval(Lf,lf_y)

    l1 = np.mean(np.absolute(Lf-lf_rec), axis=(2,3,4))
    l1[3,3] = 0

    l1y = np.mean(np.absolute(Lf-lf_y), axis=(2,3,4))
    l1y[3,3] = 0

    ss = eval_ssim(Lf[np.newaxis, ...], lf_rec[np.newaxis, ...])
    ss[3,3] = 0

    ssy = eval_ssim(Lf[np.newaxis, ...], lf_y[np.newaxis, ...])
    ssy[3,3] = 0

    return plf, plfy, l1, l1y, ss, ssy

###  **Testing**

In [0]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

datapath = '/media/flash/ExTra/ulf_focdef/data/TestSet/PAPER/' # datapath for test light fields
img_list = listdir(datapath)


img_list.sort()

EXTRA=False
FULL=True

fname = 'focdef_test/'
if not os.path.isdir(fname):
    os.mkdir(fname)
path = fname

oP = patchsize[0] # output patch size from the network
P = patchsize[0] # patch size
odP = patchsize[0] # output disparity patches
crop = 15
overlap = 15

#helper functions for patch operations
from readIlliumImages_v1 import illiumTools as tools
tools = tools()
tools.verbosity = 0
tools.p = P
tools.s = P - 2*crop - overlap
tools.op = oP
tools.coP = P - 2*crop # 30 - patch size 60, 90 - for patch size 120 
tools.angRes = lfsize[2]
tools.viewH = lfsize[0]
tools.viewW = lfsize[1]

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

with tf.Session(config = config) as sess:
    
    saver = tf.train.Saver()
    saver.restore(sess, '/media/flash/ExTra/ulf_focdef/trained_models/model.ckpt-15999') #restoring model
        
    losss = []; slosss = []
    plist = []; pylist = []
    slist = []; sylist = []
    l1list = []; l1ylist = []

    for img in img_list:
        #img = img_list[i]
        if img[-4:] != '.png':
            print('not an image')
            continue
            
        img_num = img_list.index(img)
        
        print('processing LF img', img, img_num)
        img_name = datapath+img
        
        value = tf.read_file(img_name)
        lf = tf.image.decode_png(value, channels=3, dtype=tf.uint16)
        lf = tf.to_float(lf[:lfsize[0]*14, :lfsize[1]*14, :])/65535.0
        #lf = tf.image.adjust_gamma(lf, gamma=0.4)
        #light field preprocessing
        lf = tf.image.rgb_to_hsv(tf.pow(lf,1/1.5))
        lf = tf.concat([lf[:,:,0:1],lf[:,:,1:2]*1.5,lf[:,:,2:3]],axis=2)
        lf = tf.image.hsv_to_rgb(lf)
        lf = tf.clip_by_value(lf, 0.0,1.0)
        
        lf = tf.transpose(tf.reshape(lf, [lfsize[0], 14, lfsize[1], 14, 3], name='img_lf'), [0, 2, 1, 3, 4])
        avsz = 8
        lf = lf[:, :, (14/2)-(avsz/2):(14/2)+(avsz/2), (14/2)-(avsz/2):(14/2)+(avsz/2), :]
        lf = lf[:,:,1:,1:,:]
        
        
        # permute the dim and get patches 
        Lf = tf.transpose(tf.squeeze(lf),[2,3,0,1,4])
        Lf = Lf.eval()
        
        #extracting patches from light field
        pLfs, rlist, clist = tools.extract_patches(Lf) # N,v,v P, P, 3  
        pCodedImgs = pLfs.transpose([0,3,4,1,2,5])*code
        pCodedImgs = pCodedImgs.sum(-2)
        pCodedImgs = pCodedImgs.sum(-2)

        # fwd pass of batches 
        print('fwd ...')
        batch_list = range(0,pCodedImgs.shape[0],batchsize)
        depths = []; genCv = []; genLf = []; geny = []
        #print pLfs.shape
        for bid in batch_list:
            #print pLfs[bid:bid+batchsize,3,3,3:-3,3:-3,:].shape, pCodedImgs[bid:bid+batchsize,3:-3,3:-3,:].shape, pLfs[bid:bid+batchsize,:,:,3:-3,3:-3,:].shape
            feed_dict = {x_batch: pLfs[bid:bid+batchsize,3,3,:,:,:], 
                         xc_batch: pCodedImgs[bid:bid+batchsize,:,:,:], 
                         lf_batch: pLfs[bid:bid+batchsize,:,:,:,:,:].transpose([0,3,4,1,2,5]), 
                        is_training: False}
            #x_batch: pLfs[bid:bid+batchsize,3,3,3:-3,3:-3,:],
            #Obtain losses, light fields and ray depthds 
            bray_depths, blf_shear, by, tloss, sloss = sess.run([ray_depths, lf_shear, y, train_loss, shear_loss], 
                                                         feed_dict=feed_dict)
            
            depths.append(np.array(bray_depths))
            genLf.append(np.array(blf_shear))
            geny.append(np.array(by))
            
            losss.append(tloss)
            slosss.append(sloss)
            
        depths = np.vstack(depths)
        genLf = np.vstack(genLf)
        geny = np.vstack(geny)
        
        depths = np.tile(depths[:,:,:,:,:,np.newaxis],(1,1,1,1,1,3))
        depths = depths.transpose(0,3,4,1,2,5)
        genLf = genLf.transpose(0,3,4,1,2,5)
        geny = geny.transpose(0,3,4,1,2,5)
        
        offset = crop
        offr = 9
        offc = 0
        
        #combining patches to generate full light fields and ray depths
        if FULL:
            warp_reconst = tools.combine_patches(depths, rlist, clist,1)
            lf_rec = tools.combine_patches(genLf, rlist, clist,1)
            lf_y = tools.combine_patches(geny, rlist, clist,1)
            
            Lf = Lf[:,:,offset:-offset-offr,offset:-offset-offc,:]
            warp_reconst = warp_reconst[:,:,offset:-offset-offr,offset:-offset-offc,:]
            lf_rec = lf_rec[:,:,offset:-offset-offr,offset:-offset-offc,:]
            lf_y = lf_y[:,:,offset:-offset-offr,offset:-offset-offc,:]
            
            warp_reconst = warp_reconst[:,:,7:-6,7:-6,:]
            Lf = Lf[:,:,7:-6,7:-6,:]
            print Lf.shape
            lf_rec = lf_rec[:,:,7:-6,7:-6,:]
            lf_y = lf_y[:,:,7:-6,7:-6,:]
            
            #PSNR, SSIM evaluation
            plf, plfy, l1, l1y, ss, ssy = all_eval(Lf, lf_rec, lf_y)
            
            plist.append(plf); pylist.append(plfy)  
            l1list.append(l1); l1ylist.append(l1y)
            slist.append(ss);  sylist.append(ssy)
            
            #PSNR, SSIM of unrefined and refined light fields
            print('psnr', plf.sum()/48.0, 'psnry', plfy.sum()/48.0)
            print('ssim', ss.sum()/48.0,  'ssimy', ssy.sum()/48.0)

            plt.imsave(path+str(img_num).zfill(3)+'_bcvdepth.png', warp_reconst[3,3,:,:,0].clip(-2.15,2.15), cmap='plasma')
            np.save(path+str(img_num).zfill(3)+'_bcvdepth.npy', warp_reconst[3,3,:,:,:])
            imsave(path+str(img_num).zfill(3)+'_cv.png', lf_rec[3,3,:,:,:])
            imsave(path+str(img_num).zfill(3)+'_tr.png', lf_rec[0,6,:,:,:])
            imsave(path+str(img_num).zfill(3)+'_try.png', lf_y[0,6,:,:,:])
            imsave(path+str(img_num).zfill(3)+'_trz.png', Lf[0,6,:,:,:])
            
            err = np.abs(lf_rec[0,6,:,:,:]-Lf[0,6,:,:,:])
            err = err.mean(2)
            erry = np.abs(lf_y[0,6,:,:,:]-Lf[0,6,:,:,:])
            erry = erry.mean(2)
                        
            plt.imsave(path+str(img_num).zfill(3)+'_err.png', err.clip(0,0.1), cmap='jet')
            plt.imsave(path+str(img_num).zfill(3)+'_erry.png', erry.clip(0,0.1), cmap='jet')

        if EXTRA:
            #generate only full depths
            depths = tools.combine_patches(depths[:,3,3,:,:,:], rlist, clist)
            depths = depths[offset:-offset-offr,offset:-offset-offc,:]
            depths = depths[7:-6,7:-6,:]
            np.save(path+str(img_num).zfill(3)+'_bcvdepth.npy', depths)

        codedImg = tools.combine_patches(pCodedImgs, rlist, clist)
        codedImg = codedImg[offset:-offset-offr,offset:-offset-offc,:]
        codedImg = codedImg[7:-6,7:-6,:]
        imsave(path+str(img_num).zfill(3)+'_coded.png', codedImg)
    
    losss = np.vstack(losss)
    print(losss.mean())
    losss = np.vstack(slosss)
    print(losss.mean())

INFO:tensorflow:Restoring parameters from /media/flash/ExTra/ulf_focdef/trained_models/model.ckpt-15999
processing LF img Cars.png 0
fwd ...
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
(7, 7, 323, 497, 3)


/home/flash/anaconda2/envs/tfv1p12/lib/python2.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log10


psnr 35.8107431935 psnry 35.6033650716
ssim 0.970317896268 ssimy 0.973363578135


/home/flash/anaconda2/envs/tfv1p12/lib/python2.7/site-packages/ipykernel_launcher.py:149: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/flash/anaconda2/envs/tfv1p12/lib/python2.7/site-packages/ipykernel_launcher.py:151: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/flash/anaconda2/envs/tfv1p12/lib/python2.7/site-packages/ipykernel_launcher.py:152: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/flash/anaconda2/envs/tfv1p12/lib/python2.7/site-packages/ipykernel_launcher.py:153: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


patch shape (6, 210, 210, 3)
not lf (6, 210, 210, 3) 15
not lf (6, 180, 180, 3)
patches shape (6, 180, 180, 3)
processing LF img Flower1.png 1


/home/flash/anaconda2/envs/tfv1p12/lib/python2.7/site-packages/ipykernel_launcher.py:182: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


fwd ...
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
(7, 7, 323, 497, 3)
psnr 36.1804210837 psnry 36.1888189691
ssim 0.968620180407 ssimy 0.971716505889
patch shape (6, 210, 210, 3)
not lf (6, 210, 210, 3) 15
not lf (6, 180, 180, 3)
patches shape (6, 180, 180, 3)
processing LF img Flower2.png 2
fwd ...
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
(7, 7, 323, 497, 3)
psnr 36.1954615077 psnry 35.8245765065
ssim 0.962280140585 ssimy 0.962786883115
patch shape (6, 210, 210, 3)
not lf (6, 210, 210, 3) 15
not lf (6, 180, 180, 3)
patches shape (6, 180, 180, 3)
processing LF img Rock.png 3
fwd ...
patch shape (6, 7, 7, 210, 210, 3)
patches shape (6, 49, 180, 180, 3)
pat

In [0]:
plist = np.array(plist)
pylist = np.array(pylist)
slist = np.array(slist)
sylist = np.array(sylist)
l1list = np.array(l1list)
l1ylist = np.array(l1ylist)

###  **PSNR, SSIM and L1 errors**

In [0]:
print (plist.mean(), pylist.mean())
print (slist.mean(), sylist.mean())
print (l1list.mean(), l1ylist.mean())

In [0]:
np.save(path+'plist',plist)
np.save(path+'pylist',pylist)
np.save(path+'slist',slist)
np.save(path+'sylist',sylist)
np.save(path+'l1list',l1list)
np.save(path+'l1ylist',l1ylist)